In [0]:
# Avirut Mehta
# 200321@1300

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# compiled source from other files
fips = pd.read_csv('/content/drive/My Drive/Global Health Project/Analysis/wvfips.csv', dtype={'fips': str}, index_col=[0]).to_dict()['fips']
d = pd.read_csv('/content/drive/My Drive/Global Health Project/Analysis/joint_am_200412.csv', index_col=[0,1])
d['FIPS'] = [str(val)[0:-2] for val in d['FIPS']]

## Map Analysis

In [0]:
!pip install --upgrade plotly

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.6.0)


In [0]:
import plotly.express as px
import json

In [0]:
# src: https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json
# then use filter feature on https://mapshaper.org to delete everything but desired state
geojson = open('/content/drive/My Drive/Global Health Project/Analysis/geojson-counties-fips-wv.json')
countyjson = json.load(geojson)

figconfig = {'displayModeBar': False}

In [0]:
def drawGraph(title, data, color, key):
  rangeLow = data[color].min()
  rangeHigh = data[color].max()
  scale = 'Reds'
  ccmp = None
  if rangeLow < 0 and rangeHigh > 0 and abs(rangeLow/rangeHigh) > 0.5:
    ccmp = 0
    scale = 'RdBu_r'
  wvcenter = {'lat': 38.9, 'lon': -78}
  # onhover = data.columns.values
  onhover= ['county'] if 'county' in data.columns.values else ['County']
  if 'County' in data.columns.values:
    data['FIPS'] = data['County'].map(fips)
  if 'county' in data.columns.values:
    data['FIPS'] = data['county'].map(fips)
  fig = px.choropleth_mapbox(data, geojson=countyjson, locations='FIPS', color=color,
                           title=title,
                           color_continuous_scale=scale,
                           color_continuous_midpoint=ccmp,
                           # range_color=(rangeLow, rangeHigh),
                           hover_data=onhover,
                           mapbox_style='carto-darkmatter',
                           zoom=6.3, center = wvcenter,
                           opacity=1,
                           labels={color: key}
                           )
  fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
  fig.update_layout({'paper_bgcolor': 'rgba(0,0,0,0)', 'plot_bgcolor': 'rgba(0,0,0,0)'})
  fig.update_layout(font={'color': 'rgba(255,255,255,255)'})
  fig.update_layout(xaxis={'scaleanchor': 'y', 'constrain': 'domain'}, coloraxis_colorbar_x=0.02)
  return fig

In [0]:
def getDeltas(data, startYear, endYear):
  d = data
  counties = np.unique([ind[0] for ind in d.index.values])
  f = pd.DataFrame(counties, columns=['county'])
  f['iPop'] = f['fPop'] = f['dPop'] = f['iMME'] = f['fMME'] = f['dMME'] = f['iPCI'] = f['fPCI'] = f['dPCI'] = f['iSui'] = f['fSui'] = f['dSui'] = 0
  for index, row in f.iterrows():
    county = row['county']
    f.at[index, 'iPop'] = d.loc[(county, startYear), 'PopCDC']
    f.at[index, 'fPop'] = d.loc[(county, endYear), 'PopCDC']
    f.at[index, 'iMME'] = d.loc[(county, startYear), 'TotMME']
    f.at[index, 'fMME'] = d.loc[(county, endYear), 'TotMME']
    f.at[index, 'iPCI'] = d.loc[(county, startYear), 'PerCapIncome']
    f.at[index, 'fPCI'] = d.loc[(county, endYear), 'PerCapIncome']
    f.at[index, 'iSui'] = d.loc[(county, startYear), 'Suicides']
    f.at[index, 'fSui'] = d.loc[(county, endYear), 'Suicides']
    f.at[index, 'iEmp'] = d.loc[(county, startYear), 'UnemployedRate']
    f.at[index, 'fEmp'] = d.loc[(county, endYear), 'UnemployedRate']
    f.at[index, 'dPop'] = f.at[index, 'fPop'] - f.at[index, 'iPop']
    f.at[index, 'dMME'] = f.at[index, 'fMME'] - f.at[index, 'iMME']
    f.at[index, 'dPCI'] = f.at[index, 'fPCI'] - f.at[index, 'iPCI']
    f.at[index, 'dSui'] = f.at[index, 'fSui'] - f.at[index, 'iSui']
    f.at[index, 'dEmp'] = f.at[index, 'fEmp'] - f.at[index, 'iEmp']
    pcdPop = f.at[index, 'dPop']/f.at[index, 'iPop']*100
    pcdMME = f.at[index, 'dMME']/f.at[index, 'iMME']*100
    pcdPCI = f.at[index, 'dPCI']/f.at[index, 'iPCI']*100
    pcdSui = f.at[index, 'dSui']/f.at[index, 'iSui']*100
    pcdEmp = f.at[index, 'dEmp']/f.at[index, 'iEmp']*100
    f.at[index, 'pcdPop'] = pcdPop if abs(pcdPop) < 100000 else f.at[index, 'dPop']*100
    f.at[index, 'pcdMME'] = pcdMME if abs(pcdMME) < 100000 else f.at[index, 'dMME']*100
    f.at[index, 'pcdPCI'] = pcdPCI if abs(pcdPCI) < 100000 else f.at[index, 'dPCI']*100
    f.at[index, 'pcdSui'] = pcdSui if abs(pcdSui) < 100000 else f.at[index, 'dSui']*100
    f.at[index, 'pcdEmp'] = pcdEmp if abs(pcdEmp) < 100000 else f.at[index, 'dEmp']*100
  return f

In [0]:
y2y = [getDeltas(d, startYear, startYear+1) for startYear in range(2006, 2014)]
y2ymean = pd.concat(y2y).groupby('county').mean().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning:

divide by zero encountered in long_scalars

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning:

invalid value encountered in long_scalars



In [0]:
pcdSuiY2Ymean = drawGraph('% Change in Suicides, avg. y2y 2006-2014', y2ymean, 'pcdSui', '%Δ Suicides')
pcdSuiY2Ymean.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/pcdSuiY2Ymean.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
pcdMMEY2Ymean = drawGraph('% Change in Opioids, avg. y2y 2006-2014', y2ymean, 'pcdMME', '%Δ MME')
pcdMMEY2Ymean.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/pcdMMEY2Ymean.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
pcdPCIY2Ymean = drawGraph('% Change in Per Capita Income, avg. y2y 2006-2014', y2ymean, 'pcdPCI', '%Δ Per Capita Income')
pcdPCIY2Ymean.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/pcdPCIY2Ymean.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
pcdPopY2Ymean = drawGraph('% Change in Population, avg. y2y 2006-2014', y2ymean, 'pcdPop', '%Δ Population')
pcdPopY2Ymean.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/pcdPopY2Ymean.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
pcdEmpY2Ymean = drawGraph('% Change in Employment, avg. y2y 2006-2014', y2ymean, 'pcdEmp', '%Δ Unemployment Rate')
pcdEmpY2Ymean.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/pcdEmpY2Ymean.html', config=figconfig, include_plotlyjs='cdn')

## Correlations

In [0]:
# necessary to save as image

!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [0]:
import plotly.express as px

figconfig = {'displayModeBar': False}

In [0]:
d = pd.read_csv('/content/drive/My Drive/Global Health Project/Analysis/joint_am_200412.csv')
d['FIPS'] = [str(val)[0:-2] for val in d['FIPS']]

In [5]:
d.head()

,County,Year,NumTrans,TotMME,TotDU,Suicides,PopCDC,PerCapIncome,UnemployedRate,FIPS
0,Barbour,2006,1044.0,3.220149e+06,508100.0,3.0,15922.0,22926.0,5.7,54001
1,Barbour,2007,1310.0,4.126327e+06,625150.0,2.0,16036.0,24086.0,5.5,54001
2,Barbour,2008,1632.0,5.043763e+06,766200.0,1.0,16241.0,25464.0,4.9,54001
3,Barbour,2009,1823.0,6.325642e+06,869860.0,2.0,16479.0,25686.0,8.7,54001
4,Barbour,2010,2137.0,7.092897e+06,945110.0,0.0,16589.0,26381.0,10.0,54001


In [0]:
d['SuiPerCap'] = d['Suicides']/d['PopCDC']*100000
d['MMEPerCap'] = d['TotMME']/d['PopCDC']

In [0]:
def makescatter(data, x, y, title, color=None):
  labels = {'NumTrans': '# of Opioid Transactions', 'TotMME': 'Total Milligram Morphine Equivalent', 'TotDU': 'Total Dosage Units',
          'PopCDC': 'Population', 'PerCapIncome': 'Per Capita Income', 'UnemployedRate': 'Unemployment Rate', 'SuiPerCap': 'Suicides Per 100,000',
          'MMEPerCap': 'Milligram Morphine Equivalent per Capita'}
  fig =  px.scatter(d, x=x, y=y, trendline='ols', labels=labels, color=color,
                   title=title, hover_data=['County', 'Year'])
  fig.layout.template = 'plotly_dark'
  fig.update_layout({'paper_bgcolor': 'rgba(0,0,0,0)', 'plot_bgcolor': 'rgba(0,0,0,0)'})
  return fig

In [0]:
SuiPerCapVsUnemp = makescatter(d, x='UnemployedRate', y='SuiPerCap', title='Suicides Per Capita vs. Unemployment Rate, 2006-2014', color='Year')
SuiPerCapVsUnemp.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/SuiPerCapvsUnemp.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
SuiPerCapVsMME = makescatter(d, x='MMEPerCap', y='SuiPerCap', title='Suicides per Capita vs. Opioid Distribution, 2006-2014', color='Year')
SuiPerCapVsMME.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/SuiPerCapVsMME.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
SuiPerCapVsYear = makescatter(d, x='Year', y='SuiPerCap', title='Suicides per Capita by Year and County, 2006-2014')
SuiPerCapVsYear.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/SuiPerCapVsYear.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
PopVsYear = makescatter(d, x='Year', y='PopCDC', title='County Population by Year, 2006-2014')
PopVsYear.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/PopVsYear.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
PCIVsYear = makescatter(d, x='Year', y='PerCapIncome', title='Per Capita Income by Year and County, 2006-2014')
PCIVsYear.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/PCIVsYear.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
UnempVsMME = makescatter(d, x='MMEPerCap', y='UnemployedRate', title='Unemployment Rate vs. Opioid Distribution, 2006-2014', color='Year')
UnempVsMME.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/PCIVsYear.html', config=figconfig, include_plotlyjs='cdn')

In [0]:
UnempVsYear = makescatter(d, x='Year', y='UnemployedRate', title='Unemployment Rate by Year and County, 2006-2014')
UnempVsYear.write_html('/content/drive/My Drive/Global Health Project/Analysis/0utput/UnempVsYear.html', config=figconfig, include_plotlyjs='cdn')